<h1><font color='blue'> 9E and 9F: Finding the Probability P(Y==1|X)</font></h1>

<h2><font color='Geen'> 9E: Implementing Decision Function of SVM RBF Kernel</font></h2>

<font face=' Comic Sans MS' size=3>After we train a kernel SVM model, we will be getting support vectors and their corresponsing coefficients $\alpha_{i}$

Check the documentation for better understanding of these attributes: 

https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
<img src='https://i.imgur.com/K11msU4.png' width=500>

As a part of this assignment you will be implementing the ```decision_function()``` of kernel SVM, here decision_function() means based on the value return by ```decision_function()``` model will classify the data point either as positive or negative

Ex 1: In logistic regression After traning the models with the optimal weights $w$ we get, we will find the value $\frac{1}{1+\exp(-(wx+b))}$, if this value comes out to be < 0.5 we will mark it as negative class, else its positive class

Ex 2: In Linear SVM After traning the models with the optimal weights $w$ we get, we will find the value of $sign(wx+b)$, if this value comes out to be -ve we will mark it as negative class, else its positive class.

Similarly in Kernel SVM After traning the models with the coefficients $\alpha_{i}$ we get, we will find the value of 
$sign(\sum_{i=1}^{n}(y_{i}\alpha_{i}K(x_{i},x_{q})) + intercept)$, here $K(x_{i},x_{q})$ is the RBF kernel. If this value comes out to be -ve we will mark $x_{q}$ as negative class, else its positive class.

RBF kernel is defined as: $K(x_{i},x_{q})$ = $exp(-\gamma ||x_{i} - x_{q}||^2)$

For better understanding check this link: https://scikit-learn.org/stable/modules/svm.html#svm-mathematical-formulation
</font>

## Task E

> 1. Split the data into $X_{train}$(60), $X_{cv}$(20), $X_{test}$(20)

> 2. Train $SVC(gamma=0.001, C=100.)$ on the ($X_{train}$, $y_{train}$)

> 3. Get the decision boundry values $f_{cv}$ on the $X_{cv}$ data  i.e. ` `$f_{cv}$ ```= decision_function(```$X_{cv}$```)```  <font color='red'>you need to implement this decision_function()</font>

In [2]:
import numpy as np
import math
import pandas as pd
from sklearn.datasets import make_classification
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [3]:
X, y = make_classification(n_samples=5000, n_features=5, n_redundant=2,
                           n_classes=2, weights=[0.7], class_sep=0.7, random_state=15)

### Pseudo code

clf = SVC(gamma=0.001, C=100.)<br>
clf.fit(Xtrain, ytrain)

<font color='green'>def</font> <font color='blue'>decision_function</font>(Xcv, ...): #use appropriate parameters <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color='green'>for</font> a data point $x_q$ <font color='green'>in</font> Xcv: <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color='grey'>#write code to implement $(\sum_{i=1}^{\text{all the support vectors}}(y_{i}\alpha_{i}K(x_{i},x_{q})) + intercept)$, here the values $y_i$, $\alpha_{i}$, and $intercept$ can be obtained from the trained model</font><br>
   <font color='green'>return</font> <font color='grey'><i># the decision_function output for all the data points in the Xcv</i></font>
    
fcv = decision_function(Xcv, ...)  <i># based on your requirement you can pass any other parameters </i>

<b>Note</b>: Make sure the values you get as fcv, should be equal to outputs of clf.decision_function(Xcv)


In [4]:
# you can write your code here

def K(xi,xq):
    return math.exp(-0.001 * sum([(a - b) ** 2 for a, b in zip(xq, xi)]))

#implemented decision function
def decision_function(x_cv):
    fcv = []
    for xq in x_cv:
        result = 0
        for idx,xi in enumerate(support_vectors):
            result += coef[idx]* K(xi,xq) #  (∑all the support vectors𝑖=1(𝑦𝑖𝛼𝑖𝐾(𝑥𝑖,𝑥𝑞))
        result += intercept
        fcv.append(result)
    return np.array(fcv)#converting into np array for comparision
    


    
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train,X_cv,y_train,y_cv = train_test_split(X_train,y_train,test_size=0.2,random_state=42)#cross_validation spliting

clf = SVC(gamma=0.001,C=100)
clf.fit(X_train,y_train)

coef = clf.dual_coef_.transpose()#converting into column vector
intercept = clf.intercept_
support_vectors = clf.support_vectors_


#sklearn based decision_function result
res = clf.decision_function(X_cv).reshape(len(X_cv),1)#shape of fcv is (len(xcv),1)
#custom decision_function result has shape (len(cv),1)
fcv = decision_function(X_cv)
print('Yes they are equal') if np.array_equal(fcv,res) == True else 'Not equal'
#decision_function(clf)

Yes they are equal


<h2><font color='Geen'> 9F: Implementing Platt Scaling to find P(Y==1|X)</font></h2>

Check this <a href='https://drive.google.com/open?id=133odBinMOIVb_rh_GQxxsyMRyW-Zts7a'>PDF</a>
<img src='https://i.imgur.com/CAMnVnh.png'>


## TASK F


> 4. Apply SGD algorithm with ($f_{cv}$, $y_{cv}$) and find the weight $W$ intercept $b$ ```Note: here our data is of one dimensional so we will have a one dimensional weight vector i.e W.shape (1,)``` 

> Note1: Don't forget to change the values of $y_{cv}$ as mentioned in the above image. you will calculate y+, y- based on data points in train data

> Note2: the Sklearn's SGD algorithm doesn't support the real valued outputs, you need to use the code that was done in the `'Logistic Regression with SGD and L2'` Assignment after modifying loss function, and use same parameters that used in that assignment.
<img src='https://i.imgur.com/zKYE9Oc.png'>
if Y[i] is 1, it will be replaced with y+ value else it will replaced with y- value

> 5. For a given data point from $X_{test}$, $P(Y=1|X) = \frac{1}{1+exp(-(W*f_{test}+ b))}$ where ` `$f_{test}$ ```= decision_function(```$X_{test}$```)```, W and b will be learned as metioned in the above step

__Note: in the above algorithm, the steps 2, 4 might need hyper parameter tuning, To reduce the complexity of the assignment we are excluding the hyerparameter tuning part, but intrested students can try that__


If any one wants to try other calibration algorithm istonic regression also please check these tutorials

1. http://fa.bianp.net/blog/tag/scikit-learn.html#fn:1

2. https://drive.google.com/open?id=1MzmA7QaP58RDzocB0RBmRiWfl7Co_VJ7

3. https://drive.google.com/open?id=133odBinMOIVb_rh_GQxxsyMRyW-Zts7a

4. https://stat.fandom.com/wiki/Isotonic_regression#Pool_Adjacent_Violators_Algorithm


In [5]:
N_p = np.count_nonzero(y_train)
N_n = len(y_train) - N_p
#new value of y+ and y-
y_p = (N_p + 1)/(N_p+2)
y_n = 1/(N_n+2)

#replacing y_cv with y_p where it is 1 and y_n where it is 0
y_cv = np.where(y_cv ==0, y_n, y_cv) 
y_cv = np.where(y_cv ==1,y_p,y_cv)




In [6]:
w = np.zeros(1)
b = 0
eta0  = 0.0001
alpha = 0.0001
N = len(fcv)
w.shape

(1,)

In [7]:
def sigmoid(w,x,b):
    return 1/(1 + np.exp(-(w*x + b)))

In [17]:
def compute_log_loss(w,x,y,b):
    #sigmoid returns the probability scores 
    score = sigmoid(w,x,b)
    n = len(y)
    loss = 0
    for i in range(n):
        loss += (y[i]*np.log10(score[i]) + (1-y[i])*np.log10(1-score[i]))
    loss = -(1/n)*loss
    return loss

In [19]:
def LR_SGD(x_train,y_tr,w,b,eta0,alpha,epoch,N):
    log_loss = [] # creating log_loss empty list
    #iterating for each epoch to find the loss
    for epch in range(epoch):
        #iterating on each training data point to update the weights
        for i in range(N):
            #updating w
            w = ((1 - eta0*(alpha/N))*w)  + ((alpha*x_train[i])*(y_tr[i] - sigmoid(w,x_train[i],b)))
            #updating b
            b = b + eta0*(y_tr[i] - sigmoid(w,x_train[i],b))
        
        #computing log_loss for train data on this epoch
        loss_train = compute_log_loss(w,x_train,y_tr,b)
        log_loss.append(loss_train)
        #checking change in loss for convergence
        if(abs(loss_train-log_loss[-1]) < 0.001):
            break
    return w,b

In [20]:
epoch = 10
#getting the W and b
w,b = LR_SGD(fcv,y_cv,w,b,eta0,alpha,epoch,N)

In [21]:
print(w,b)

[0.53746424] [-0.09766821]


In [22]:
f_test = decision_function(X_test)

In [23]:
Prob_X_test = sigmoid(w,f_test,b)

In [31]:
Prob_X_test[:5]

array([[0.29939726],
       [0.30877606],
       [0.38878954],
       [0.19038363],
       [0.11714493]])